In [2]:
# Start with loading all necessary libraries
import numpy as np
import pandas as pd
import pyodbc

import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [4]:
sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=L_AAGDATIX;'
                            'DATABASE=DatixCRM;'
                            'Trusted_Connection=yes') 
query = "set transaction isolation level read uncommitted select inc_organisation,inc_locactual,inc_unit,inc_specialty,inc_loctype,inc_result,inc_severity,show_other_contacts,show_employee,show_witness,show_document,inc_reportedby,inc_notes from DatixCRM.dbo.incidents_main where inc_type='PAT' and inc_category='MEDIC'"
df = pd.read_sql(query, sql_conn)
df.head()

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'SQL Server' : file not found (0) (SQLDriverConnect)")

In [ ]:
df['division'] = df['inc_unit'].str[:3]
df['care group'] = df['inc_unit'].str[3:6]
df = df.drop('inc_unit',axis=1)

In [6]:
category_columns = list(df.columns.values)
category_columns.remove('inc_notes')
count_accuracies = []
tfidf_accuracies = []
for column in category_columns:
    df = df.dropna(subset=[column])
    y = df[column]
    X_train, X_test, y_train, y_test = train_test_split(
                                    df['inc_notes'], y,
                                    test_size=0.33,
                                    random_state=53)
    count_vectorizer = CountVectorizer(stop_words='english')
    count_train = count_vectorizer.fit_transform(X_train.values)
    count_test = count_vectorizer.transform(X_test.values)
    # Print the first 10 features of the count_vectorizer
    #print(count_vectorizer.get_feature_names()[:10])


    # Initialize a TfidfVectorizer object: tfidf_vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words="english",max_df=0.7)

    # Transform the training data: tfidf_train 
    tfidf_train = tfidf_vectorizer.fit_transform(X_train.values)

    # Transform the test data: tfidf_test 
    tfidf_test = tfidf_vectorizer.transform(X_test.values)

    # Print the first 10 features
    #print(tfidf_vectorizer.get_feature_names()[:10])

    # Print the first 5 vectors of the tfidf training data
    #print(tfidf_train.A[:5])


    # Create the CountVectorizer DataFrame: count_df
    #count_df = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())

    # Create the TfidfVectorizer DataFrame: tfidf_df
    #tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())

    # Print the head of count_df
    #print(count_df.head())

    # Print the head of tfidf_df
    #print(tfidf_df.head())

    # Calculate the difference in columns: difference
    #difference = set(count_df.columns) - set(tfidf_df.columns)
    #print(difference)

    # Check whether the DataFrames are equal
    #print(count_df.equals(tfidf_df))


    count_nb_classifier = MultinomialNB()
    count_nb_classifier.fit(count_train, y_train)
    count_pred = nb_classifier.predict(count_test)
    count_accuracies.append(100*metrics.accuracy_score(y_test,count_pred))
    
    tfidf_nb_classifier = MultinomialNB()
    tfidf_nb_classifier.fit(tfidf_train, y_train)
    tfidf_pred = nb_classifier.predict(tfidf_test)
    tfidf_accuracies.append(100*metrics.accuracy_score(y_test,tfidf_pred))    

NameError: name 'df' is not defined

In [7]:
count_from50 = [int(round(accuracy/2)) for accuracy in count_accuracies]
n_category_columns = len(category_columns)
colors = [['green' if count_from50[j]>i else 'red' for j in range(n_category_columns)] for i in range(50)]
category_columns = [column.replace('inc_organisation','site') for column in category_columns]
category_columns = [column.replace('inc_locactual','ward/dept/unit') for column in category_columns]
category_columns = [column.replace('inc_unit','division & care group') for column in category_columns]
category_columns = [column.replace('inc_specialty','specialty') for column in category_columns]
category_columns = [column.replace('inc_loctype','location type') for column in category_columns]
category_columns = [column.replace('inc_result','result') for column in category_columns]
category_columns = [column.replace('inc_severity','severity') for column in category_columns]
category_columns = [column.replace('inc_reportedby','reported by') for column in category_columns]
category_columns = [column.replace('show_other_contacts','other patients involved?') for column in category_columns]
category_columns = [column.replace('show_employee','other employees involved?') for column in category_columns]
category_columns = [column.replace('show_witness','any witnesses?') for column in category_columns]
category_columns = [column.replace('show_document','any documents attached?') for column in category_columns]
for i in range(50):
    plt.scatter(x=np.ones(len(category_columns))*(i+1),y=category_columns,color=colors[i])
plt.xlim((0,51));
fig1 = plt.figure(1)
fig1.text(0, 0.95, "Correct", ha="center", va="bottom", size="large", color="green");
fig1.text(0.06, 0.95, "/", ha="center", va="bottom", size="large");
fig1.text(0.13,0.95,"Incorrect", ha="center", va="bottom", size="large", color="red");
fig1.text(0.19, 0.95, " prediction from freetext entry for medicine patient safety events", va="bottom", size="large");

NameError: name 'accuracies' is not defined

In [ ]:
tfidf_from50 = [int(round(accuracy/2)) for accuracy in tfidf_accuracies]
colors = [['green' if tfidf_from50[j]>i else 'red' for j in range(n_category_columns)] for i in range(50)]
for i in range(50):
    plt.scatter(x=np.ones(len(category_columns))*(i+1),y=category_columns,color=colors[i])
plt.xlim((0,51));
fig1 = plt.figure(1)
fig1.text(0, 0.95, "Correct", ha="center", va="bottom", size="large", color="green");
fig1.text(0.06, 0.95, "/", ha="center", va="bottom", size="large");
fig1.text(0.13,0.95,"Incorrect", ha="center", va="bottom", size="large", color="red");
fig1.text(0.19, 0.95, " prediction from freetext entry for medicine patient safety events", va="bottom", size="large");

In [5]:
for column in category_columns:
    classes = [str(i) for i in df[column].value_counts().index]
    cm = metrics.confusion_matrix(y_test, count_pred, labels=classes)
    
    # https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    fig, ax = plt.subplots()
    im = ax.imshow(cm, cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title='True vs Predicted label from freetext of medicine patient safety events',
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

NameError: name 'category_columns' is not defined

In [ ]:
for column in category_columns:
    classes = [str(i) for i in df[column].value_counts().index]
    cm = metrics.confusion_matrix(y_test, tfidf_pred, labels=classes)
    
    # https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    fig, ax = plt.subplots()
    im = ax.imshow(cm, cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title='True vs Predicted label from freetext of medicine patient safety events',
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

In [ ]:
# Create the list of alphas: alphas
alphas = np.arange(0,1,0.1)

# Define train_and_predict()
def count_train_and_predict(column,alpha):
    df = df.dropna(subset=[column])
    y = df[column]
    X_train, X_test, y_train, y_test = train_test_split(
                                    df['inc_notes'], y,
                                    test_size=0.33,
                                    random_state=53)

    # Initialize a CountVectorizer object: count_vectorizer
    count_vectorizer = CountVectorizer(stop_words="english")

    # Transform the training data: count_train 
    count_train = count_vectorizer.fit_transform(X_train.values)

    # Transform the test data: count_test 
    count_test = count_vectorizer.transform(X_test.values)
    
    # Instantiate the classifier: nb_classifier
    nb_classifier = MultinomialNB(alpha=alpha)
    # Fit to the training data
    nb_classifier.fit(count_train, y_train)
    # Predict the labels: pred
    pred = nb_classifier.predict(count_test)
    # Compute accuracy: score
    score = metrics.accuracy_score(y_test,pred)
    
    return score

In [ ]:
# Iterate over the alphas and print the corresponding score
for alpha in alphas:
    print('Alpha: ', alpha)
    for column in category_columns:
        print('Column: ', column, ' Score: ', count_train_and_predict(column,alpha))
        print()

In [ ]:
# Create the list of alphas: alphas
alphas = np.arange(0,1,0.1)

# Define train_and_predict()
def tfidf_train_and_predict(column,alpha):
    df = df.dropna(subset=[column])
    y = df[column]
    X_train, X_test, y_train, y_test = train_test_split(
                                    df['inc_notes'], y,
                                    test_size=0.33,
                                    random_state=53)

    # Initialize a TfidfVectorizer object: tfidf_vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words="english",max_df=0.7)

    # Transform the training data: tfidf_train 
    tfidf_train = tfidf_vectorizer.fit_transform(X_train.values)

    # Transform the test data: tfidf_test 
    tfidf_test = tfidf_vectorizer.transform(X_test.values)
    
    # Instantiate the classifier: nb_classifier
    nb_classifier = MultinomialNB(alpha=alpha)
    # Fit to the training data
    nb_classifier.fit(tfidf_train, y_train)
    # Predict the labels: pred
    pred = nb_classifier.predict(tfidf_test)
    # Compute accuracy: score
    score = metrics.accuracy_score(y_test,pred)
    
    # Get the class labels: class_labels
    class_labels = nb_classifier.classes_

    # Extract the features: feature_names
    feature_names = tfidf_vectorizer.get_feature_names()

    # Zip the feature names together with the coefficient array and sort by weights: feat_with_weights
    feat_with_weights = sorted(zip(nb_classifier.coef_[0], feature_names))

    # Print the first class label and the bottom 20  feat_with_weights entries
    for i in range(len(class_labels)):
        print(class_labels[i], feat_with_weights[-20:])
    
    return score

In [ ]:
# Iterate over the alphas and print the corresponding score
for alpha in alphas:
    print('Alpha: ', alpha)
    for column in category_columns:
        print('Column: ', column, ' Score: ', tfidf_train_and_predict(column,alpha))
        print()